# LLM Fine-tuning BERT for Text Classification Pipeline Notebook

<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/fine-tune-bert-sentiment-classifier/blob/main/bert-fine-tune.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

GitHub repository: https://github.com/unionai-oss/fine-tune-bert-sentiment-classifier/

This notebook is a pipeline for fine-tuning a pre-trained BERT model for text classification using the Hugging Face Trasnforers library.

## Project Setup:

Sign up for Union while libraries are installing below:

- Union sign up: https://signup.union.ai/
- Union Platform: https://serverless.union.ai/
- Hugging Face Sign up: https://huggingface.co/

Install python libraries by running the code cell below:

In [12]:
# Install required packages
%%bash
pip install union==0.1.64 \
            transformers==4.39.1 \
            datasets==2.18.0 \
            matplotlib==3.8.3 \
            torch==2.0.1 \
            accelerate==0.27.2 \
            scikit-learn==1.5.1

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ pip install union==0.1.64 \                                                                                     │
│     ▲                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

Auth Union accont

In [ ]:
!union create login --auth device-flow

## Build a Simple Workflow

In [ ]:
%%writefile simple_wf.py

# Import libraries and modules
# task
from flytekit import task, workflow

@task
def hello_world(name: str) -> str:
    return f"Hello {name}"

# workflow
@workflow
def main(name: str) -> str:
    return hello_world(name=name)

In [ ]:
# Run locally
!union run simple_wf.py main --name Flyte

In [ ]:
# Run on Union
!union run --remote simple_wf.py main --name Flyte

In [ ]:
!union run simple_wf.py main --help

In [ ]:
!union run --help

## Fine tune BERT for Sentiment Classification Workflow
- Create a Container Image
- Download Dataset
- Download Pre-trained BERT Model
- Visualize dataset
- Fine-tune BERT Model
- Evaluate BERT Model
- Save Model to Hugging Face Model Hub
- Use Model for Inference on New Data

Get Hugging Face API token (Read & Write Access)

https://huggingface.co/

In [ ]:
# Secrets in Union
# Add Hugging Face Secret
!union create secret hf_token

In [ ]:
!union get secret

In [ ]:
# !union delete secret hf_token


In [ ]:
%%writefile bert-fine-tune.py

# Note: this workflow can be broken into modular python files for better organization.


# %% import libraries & Create container image
# ---------------------------
from pathlib import Path
import os
from flytekit import (Deck, ImageSpec, Resources, Secret, current_context,
                      task, workflow)
from flytekit.types.directory import FlyteDirectory
from flytekit.types.file import FlyteFile
from transformers import BertForSequenceClassification

image = ImageSpec(
    packages=[
        "union==0.1.64",
        "transformers==4.39.1",
        "datasets==2.18.0",
        "matplotlib==3.8.3",
        "torch==2.0.1",
        "accelerate==0.27.2",
        "scikit-learn==1.5.1"
    ],
)

# %% download dataset
# ---------------------------
@task(
    container_image=image,
    cache=True,
    cache_version="v3",
    requests=Resources(cpu="2", mem="2Gi"),
)
def download_dataset() -> FlyteDirectory:
    from datasets import load_dataset
   
    working_dir = Path(current_context().working_directory)
    dataset_cache_dir = working_dir / "dataset_cache"

    load_dataset("imdb", cache_dir=dataset_cache_dir)

    return dataset_cache_dir

# %% download model
# ---------------------------
@task(
    container_image=image,
    cache=True,
    cache_version="v3",
    requests=Resources(cpu="2", mem="2Gi"),
)
def download_model(model: str) -> FlyteDirectory:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification

    working_dir = Path(current_context().working_directory)
    model_cache_dir = working_dir / "model_cache"

    AutoTokenizer.from_pretrained(model, cache_dir=model_cache_dir)
    AutoModelForSequenceClassification.from_pretrained(model, cache_dir=model_cache_dir)
    return model_cache_dir


# %% visualize data
# ---------------------------
@task(
    container_image=image,
    enable_deck=True,
    requests=Resources(cpu="2", mem="2Gi"),
)
def visualize_data(dataset_cache_dir: FlyteDirectory):
    from datasets import load_dataset
    import matplotlib.pyplot as plt
    import pandas as pd
    import base64
    from textwrap import dedent

    ctx = current_context()

    # Load the dataset
    dataset = load_dataset("imdb", cache_dir=dataset_cache_dir)
    train_df = pd.DataFrame(dataset['train'])
    test_df = pd.DataFrame(dataset['test'])

    # Create the deck for visualization
    deck = Deck("Dataset Analysis")

    # Sample one review from each class (positive and negative) from the training and test datasets
    train_positive_review = train_df[train_df['label'] == 1].iloc[0]['text']
    train_negative_review = train_df[train_df['label'] == 0].iloc[0]['text']
    test_positive_review = test_df[test_df['label'] == 1].iloc[0]['text']
    test_negative_review = test_df[test_df['label'] == 0].iloc[0]['text']

    # Visualize label distribution for training data
    plt.figure(figsize=(10, 5))
    train_df['label'].value_counts().plot(kind='bar', color='skyblue')
    plt.title('Train Data Label Distribution')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.tight_layout()
    train_label_dist_path = "/tmp/train_label_distribution.png"
    plt.savefig(train_label_dist_path)
    plt.close()

    # Visualize label distribution for test data
    plt.figure(figsize=(10, 5))
    test_df['label'].value_counts().plot(kind='bar', color='lightgreen')
    plt.title('Test Data Label Distribution')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.tight_layout()
    test_label_dist_path = "/tmp/test_label_distribution.png"
    plt.savefig(test_label_dist_path)
    plt.close()

    # Convert images to base64 and embed in HTML
    def image_to_base64(image_path):
        with open(image_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")

    train_image_base64 = image_to_base64(train_label_dist_path)
    test_image_base64 = image_to_base64(test_label_dist_path)

    # HTML report with styled text, tables, and embedded images
    html_report = dedent(f"""
    <div style="font-family: Arial, sans-serif; line-height: 1.6;">
        <h2 style="color: #2C3E50;">Dataset Analysis</h2>
        
        <h3 style="color: #2980B9;">Training Data Summary</h3>
        <p>Below is a summary of the training dataset including the distribution of labels.</p>
        Shape: {train_df.shape} <br>
        Columns: {train_df.columns} <br>
        Label Distribution: {train_df['label'].value_counts()} <br>
        
        <h3 style="color: #2980B9;">Sample Reviews from Training Data</h3>
        <p><strong>Positive Review:</strong> {train_positive_review}</p>
        <p><strong>Negative Review:</strong> {train_negative_review}</p>

        <h3 style="color: #2980B9;">Training Data Label Distribution</h3>
        <p>The following bar chart shows the distribution of labels in the training dataset:</p>
        <img src="data:image/png;base64,{train_image_base64}" alt="Train Data Label Distribution" width="600">

        <h3 style="color: #2980B9;">Test Data Summary</h3>
        <p>Below is a summary of the test dataset including the distribution of labels.</p>
        Shape: {test_df.shape} <br>
        Columns: {test_df.columns} <br>
        Label Distribution: {test_df['label'].value_counts()} <br>
        
        <h3 style="color: #2980B9;">Sample Reviews from Test Data</h3>
        <p><strong>Positive Review:</strong> {test_positive_review}</p>
        <p><strong>Negative Review:</strong> {test_negative_review}</p>

        <h3 style="color: #2980B9;">Test Data Label Distribution</h3>
        <p>The following bar chart shows the distribution of labels in the test dataset:</p>
        <img src="data:image/png;base64,{test_image_base64}" alt="Test Data Label Distribution" width="600">
    </div>
    """)

    # Append HTML content to the deck
    deck.append(html_report)

    # Insert the deck into the context
    ctx.decks.insert(0, deck)


# %% train model
# ---------------------------
@task(
    container_image=image,
    requests=Resources(cpu="4", mem="12Gi", gpu="1"),
)
def train_model(model_name: str, 
                dataset_cache_dir: FlyteDirectory,
                model_cache_dir: FlyteDirectory,
                epochs: int = 3
    ) -> BertForSequenceClassification:
    from datasets import load_dataset
    import numpy as np
    from transformers import(
        AutoTokenizer,
        AutoModelForSequenceClassification,
        TrainingArguments,
        Trainer,
    )
    ctx = current_context()

    working_dir = Path(ctx.working_directory)
    train_dir = working_dir / "models"

    dataset = load_dataset("imdb", cache_dir=dataset_cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=model_cache_dir)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        id2label={0: "NEGATIVE", 1: "POSITIVE"},
        label2id={"NEGATIVE": 0, "POSITIVE": 1},
        cache_dir=model_cache_dir,
    )

    def tokenizer_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    # Use a small subset such that finetuning completes
    small_train_dataset = (
        dataset["train"].shuffle(seed=42).select(range(500)).map(tokenizer_function)
    )
    small_eval_dataset = (
        dataset["test"].shuffle(seed=42).select(range(100)).map(tokenizer_function)
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return {"accuracy": np.mean(predictions == labels)}
    
    training_args = TrainingArguments(
        output_dir=train_dir,
        evaluation_strategy="epoch",
        num_train_epochs=epochs,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.train()

    return model

# %% Evaluate model
# ---------------------------
@task(
    container_image=image,
    enable_deck=True,
    requests=Resources(cpu="2", mem="12Gi", gpu="1"),
)
def evaluate_model(
    model: BertForSequenceClassification,
    dataset_cache_dir: FlyteDirectory,
    model_cache_dir: FlyteDirectory,
) -> dict:
    from datasets import load_dataset
    from transformers import AutoTokenizer, Trainer, TrainingArguments
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    import numpy as np
    import torch

    # Load the test dataset and tokenizer
    dataset = load_dataset("imdb", cache_dir=dataset_cache_dir)
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir=model_cache_dir)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    # Use a small subset (200 examples) for evaluation
    eval_dataset = dataset["test"].shuffle(seed=42).select(range(200)).map(tokenize_function)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        accuracy = accuracy_score(labels, predictions)
        f1 = f1_score(labels, predictions, average="weighted")
        precision = precision_score(labels, predictions, average="weighted")
        recall = recall_score(labels, predictions, average="weighted")
        return {
            "accuracy": accuracy,
            "f1": f1,
            "precision": precision,
            "recall": recall,
        }

    # Initialize Trainer for evaluation
    training_args = TrainingArguments(
        output_dir=".", 
        per_device_eval_batch_size=16, 
        dataloader_drop_last=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    # Run evaluation
    eval_results = trainer.evaluate()

    print(f"Evaluation results on 100 examples: {eval_results}")

    return eval_results


# %% save model
# ---------------------------
@task(
    container_image=image,
    requests=Resources(cpu="2", mem="2Gi"),
    secret_requests=[Secret(group=None, key="hf_token")],
)
def save_model(model: BertForSequenceClassification, repo_name: str) -> str:
    from huggingface_hub import HfApi

    ctx = current_context()
    
    working_dir = Path(ctx.working_directory)
    model_path = working_dir / "model"
    model.save_pretrained(model_path)

    # Ensure the model files are saved
    model_files = ["model.safetensors", "config.json"]
    for file_name in model_files:
        file_path = model_path / file_name
        if not file_path.exists():
            raise FileNotFoundError(f"Model file not found: {file_path}")
    
    # dir list for debug
    dir_list = os.listdir(model_path)
    print("Files and directories in '", model_path, "' :")
    print(dir_list)


     # set hf_token from local or union secret
    hf_token = os.getenv("HF_TOKEN")
    if hf_token is None:
        # If HF_TOKEN is not found, attempt to get it from the Flyte secrets
        hf_token = ctx.secrets.get(key="hf_token")
        print("Using Hugging Face token from Union secrets.")
    else:
        print("Using Hugging Face token from env.")

    # Create a new repository (if it doesn't exist)
    api = HfApi()
    api.create_repo(repo_name, token=hf_token, exist_ok=True)

    # Upload the model to the HF repository
    # Upload each model file to the HF repository
    for file_name in model_files:
        file_path = model_path / file_name
        api.upload_file(
            path_or_fileobj=str(file_path),
            path_in_repo=file_name,
            repo_id=repo_name,
            commit_message=f"Upload {file_name}",
            repo_type=None,
            token=hf_token
        )
    return f"Model uploaded to Hugging Face Hub: {repo_name}"

# %% Predict sentiment
# ---------------------------
@task(
    container_image=image,
    requests=Resources(cpu="2", mem="2Gi", gpu="1"),
    retries=2,
)
def predict_sentiment(model: BertForSequenceClassification, text: str, model_cache_dir: FlyteDirectory) -> dict:
    from transformers import AutoTokenizer, pipeline

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir=model_cache_dir)

    # Initialize the pipeline for sentiment analysis
    nlp_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

    # Perform the prediction
    prediction = nlp_pipeline(text)

    return prediction[0] 

# %% Workflow
@workflow
def bert_ft(model: str = "bert-base-uncased",
            repo_name: str = "my-model",
            test_text: str = "I love this movie!",
            epochs: int = 3
            ) -> (BertForSequenceClassification, FlyteDirectory, dict, dict):
    
    dataset_cache_dir = download_dataset()
    model_cache_dir = download_model(model)
    visualize_data(dataset_cache_dir=dataset_cache_dir)
    model = train_model(model_name=model, 
                        dataset_cache_dir=dataset_cache_dir, 
                        model_cache_dir=model_cache_dir,
                        epochs=epochs)
    eval_results = evaluate_model(model=model, 
                                  dataset_cache_dir=dataset_cache_dir, 
                                  model_cache_dir=model_cache_dir)
    save_model(model=model, repo_name=repo_name)
    
    # Pass the model and other needed arguments directly to the predict_sentiment task
    prediction = predict_sentiment(model=model, text=test_text, model_cache_dir=model_cache_dir)
    
    # Return results as a dict
    return model, model_cache_dir, eval_results, prediction

Run the workflow on Union with the `--remote` flag.

Replace HFUSERNAME/REPOID with your Hugging Face username and repository ID you want to save the model to. 

In [ ]:
!union run --remote bert-fine-tune.py bert_ft --repo_name HFUSERNAME/REPOID

Try running the workflow with different epochs.  Or relaunch the workflow from the UI with different hyperparameters.

```bash

!union run --remote bert-fine-tune.py bert_ft --epochs 6 --repo_name HFUSERNAME/REPOID

```

You can also modify the workflow to include different hyperparameters or models.



In [ ]:
!union run --remote bert-fine-tune.py bert_ft --epochs 6 --repo_name HFUSERNAME/REPOID


## Using the Model for Inference

There are several ways to use the model for inference from here. 

- Download the model from the Hugging Face Model Hub and use it in your own code.
- Deploy the model on a platform like Sagemaker, or Hugging Face spaces that has compute resources.
- Use the model directly in Union serverless. Run from API, or UI

### Use UnionRemote() to run the workflow on Union from the API

#### Without Union Artifacts()

In [97]:
# initialize remote context
from union.remote import UnionRemote
remote = UnionRemote()

In [98]:
# search for the 10 most recent executions of the workflow
recent_executions = remote.recent_executions(limit=10)
executions = [
    e for e in recent_executions if e.spec.launch_plan.name == "bert-fine-tune.bert_ft"
]

In [99]:
executions

[[Flyte Serialized object: Type: <FlyteWorkflowExecution> Value: <id { project: "default" domain: "development" name: "fc5519022f1a44c9390c" } spec { launch_plan { resource_type: LAUNCH_PLAN project: "default" domain: "development" name: "bert-fine-tune.bert_ft" version: "lSB3Q7nT4OniBMZzxA8aYQ" } metadata { principal: "00ugcrzo37k3huEiU5d7" system_metadata { execution_cluster: "union-us-central1" } } disable_all: false labels { } annotations { } auth_role { } } closure { outputs { uri: "s3://unionai-production-serverless-us-east-2/metadata/sagecodes/default/development/fc5519022f1a44c9390c/offloaded_outputs" } phase: SUCCEEDED started_at { seconds: 1724885764 nanos: 99438000 } duration { seconds: 545 nanos: 362008000 } created_at { seconds: 1724885762 nanos: 795334000 } updated_at { seconds: 1724886309 nanos: 461446000 } }>],
 [Flyte Serialized object: Type: <FlyteWorkflowExecution> Value: <id { project: "default" domain: "development" name: "f74143f396d744883b90" } spec { launch_plan

In [100]:
recent_ex_id = executions[0].id.name
print(recent_ex_id)

fc5519022f1a44c9390c


In [101]:
execution = remote.fetch_execution(name=recent_ex_id)

In [106]:
execution


[Flyte Serialized object: Type: <FlyteWorkflowExecution> Value: <id { project: "default" domain: "development" name: "fc5519022f1a44c9390c" } spec { launch_plan { resource_type: LAUNCH_PLAN project: "default" domain: "development" name: "bert-fine-tune.bert_ft" version: "lSB3Q7nT4OniBMZzxA8aYQ" } metadata { principal: "00ugcrzo37k3huEiU5d7" system_metadata { execution_cluster: "union-us-central1" } } disable_all: false labels { } annotations { } auth_role { } } closure { outputs { uri: "s3://unionai-production-serverless-us-east-2/metadata/sagecodes/default/development/fc5519022f1a44c9390c/offloaded_outputs" } phase: SUCCEEDED started_at { seconds: 1724885764 nanos: 99438000 } duration { seconds: 545 nanos: 362008000 } created_at { seconds: 1724885762 nanos: 795334000 } updated_at { seconds: 1724886309 nanos: 461446000 } }>]

In [107]:
execution.outputs

Partially converted to native values, call get(key, <type_hint>) to convert rest...
{
  o2: 
      [Flyte Serialized object: Type: <Literal> Value: <scalar { generic { fields { key: "eval_steps_per_second" value { number_value: 1.827 } } fields { key: "eval_samples_per_second" value { number_value: 28.101 } } fields { key: "eval_runtime" value { number_value: 7.1172 } } fields { key: "eval_recall" value { number_value: 0.83 } } fields { key: "eval_precision" value { number_value: 0.83266025641025632 } } fields { key: "eval_loss" value { number_value: 0.70779639482498169 } } fields { key: "eval_f1" value { number_value: 0.83 } } fields { key: "eval_accuracy" value { number_value: 0.83 } } } } metadata { key: "format" value: "json" }>]o0: /var/folders/nv/hcrpygqd6xvd6m2cf6w3pbvc0000gn/T/flytegj8cktz6/control_plane_metadata/local_flytekit/c1a19e5a3dfa8c499de2431da64dba75/3e02a8da35c1fd305062409a5e3e6975.pt
  o3: 
      [Flyte Serialized object: Type: <Literal> Value: <scalar { generic { f

In [109]:
ft_llm = execution.outputs["o0"].remote_source
model_cache = execution.outputs["o1"].remote_source
print(ft_llm)
print(model_cache)

union:///82/sagecodes/development/default/fc5519022f1a44c9390c-n3-0/45a729588d16119b12f29c8b68f7ab33/3e02a8da35c1fd305062409a5e3e6975.pt
union:///5a/sagecodes/development/default/f7b2094e4b7ac4536bf8-n1-0/9add4e58ea8a6635b3673087a87e96dc


In [110]:
# fetch a task
predict_task = remote.fetch_task(name="bert-fine-tune.predict_sentiment")

# Fecth specific version of a task
# task = remote.fetch_task(name="bert-fine-tune.predict_sentiment", version="_XyPQzsykVFTceNWitQmAg")

In [111]:
# view task details
predict_task

[Flyte Serialized object: Type: <FlyteTask> Value: <template { id { resource_type: TASK project: "default" domain: "development" name: "bert-fine-tune.predict_sentiment" version: "lSB3Q7nT4OniBMZzxA8aYQ" } type: "python-task" metadata { runtime { type: FLYTE_SDK version: "1.13.4" flavor: "python" } retries { retries: 2 } } interface { inputs { variables { key: "text" value { type { simple: STRING } description: "text" } } variables { key: "model" value { type { blob { format: "PyTorchModule" } } description: "model" } } variables { key: "model_cache_dir" value { type { blob { dimensionality: MULTIPART } } description: "model_cache_dir" } } } outputs { variables { key: "o0" value { type { simple: STRUCT } description: "o0" } } } } container { image: "cr.union.ai/v1/sagecodes/flytekit:E_nzXV8zZR_jSvr_pJjtbw" args: "pyflyte-fast-execute" args: "--additional-distribution" args: "unionmeta://opta-gcp-serverless-1-union-us-central1/sagecodes/default/development/BV2SVWV2TQQHEQME2CO6BQXPHI====

In [117]:
from flytekit.types.file import FlyteFile
inputs = {
    "text": "Why did this have 40 minutes of farming wheat?", 
    "model_cache_dir": model_cache,
    "model": FlyteFile(ft_llm)
}

# # Execute the task
execution = remote.execute(predict_task, inputs=inputs)
# execution = remote.execute(task, inputs=inputs, wait=True) # wait for execution to finish

url = remote.generate_console_url(execution)
print(f"🚀 Union Serverless execution url: {url}")

🚀 Union Serverless execution url: https://serverless.union.ai/org/sagecodes/projects/default/domains/development/executions/f365f3619c0cc4613aaa


In [118]:
synced_execution = remote.sync(execution)

In [119]:
synced_execution

[Flyte Serialized object: Type: <FlyteWorkflowExecution> Value: <id { project: "default" domain: "development" name: "f365f3619c0cc4613aaa" } spec { launch_plan { resource_type: TASK project: "default" domain: "development" name: "bert-fine-tune.predict_sentiment" version: "lSB3Q7nT4OniBMZzxA8aYQ" } metadata { principal: "00ugcrzo37k3huEiU5d7" system_metadata { } } notifications { } labels { } annotations { } auth_role { } } closure { outputs { uri: "s3://unionai-production-serverless-us-east-2/metadata/sagecodes/default/development/f365f3619c0cc4613aaa/offloaded_outputs" } phase: SUCCEEDED started_at { seconds: 1724886665 nanos: 186817000 } duration { seconds: 38 nanos: 541824000 } created_at { seconds: 1724886664 nanos: 165111000 } updated_at { seconds: 1724886703 nanos: 728641000 } }>]

In [120]:
outputs = synced_execution.outputs["o0"]
print(outputs)

{'score': 0.9657120108604431, 'label': 'NEGATIVE'}


If doing something like this for production, I'd recommend:
- Using Union Artifacts to store the model and dataset
- Making inference a separate workflow

Resources:
- [RAG on Union](https://github.com/unionai-oss/union-rag) - [Video tutorial](https://www.youtube.com/watch?v=IpAa1_oHNWs)
- [a100s on Union Serverless ](https://www.union.ai/blog-post/union-serverless-broadens-support-for-nvidia-accelerated-computing)
- [How LinkedIn uses Flyte for all LLM workflows](https://www.youtube.com/live/kl-fMSe2_l8?si=hEBV-LYB_SrY_1Au&t=217)